In [1]:
import pandas as pd
import json

In [2]:
date = '2017-08-30'
fn = 'DB-backup/%s/cyclings_vid1.json' % date
data = json.load(open(fn))
dfs = {}
for i, item in enumerate(data):
    if item['type']=='table':
        name = item['name']
        if name not in ( 'video2seg_temp'):
            print(i,item['name'])
            dfs[name] = pd.DataFrame(item['data'])

2 loginLog
3 Rating
4 RoadSegment
5 Users
6 Video
8 VideoRoadSeg


In [3]:
users = dfs['Users']
ratings = dfs['Rating'].drop('email', axis=1)
videos=dfs['Video']
videos.URL = videos.URL.apply(lambda x: 'https://www.youtube.com/watch?v='+x)
vid2seg = dfs['VideoRoadSeg']
segs = dfs['RoadSegment']

In [4]:
nontest_users = users[~users.email.str.contains('test')]
nontest_users_id = nontest_users.user_id
print('with test', ratings.shape)
ratings = ratings[ratings.uid.isin(nontest_users_id)]
print('without test users', ratings.shape)

with test (409, 12)
without test users (409, 12)


In [5]:
import os

In [6]:
def get_dir_fn(seed, frac):
    directory = 'downsample_experiment/seed_%d' % seed
    file_name = '%s/y_ugr_all_frac_%.2f.csv' % (directory, frac)
    os.makedirs(directory, exist_ok=True)
    return directory, file_name

In [8]:
for seed in [0, 100, 972, 5258, 7821, 40918, 57852, 168352, 291592, 789729423]:
    for frac in [0.2, 0.4, 0.6, 0.8]:
        sampled_ratings = ratings.sample(frac=frac, random_state=seed)
        directory, file_name = get_dir_fn(seed, frac)
        with open(directory+'/frac_%.2f=%d.txt' % (frac,sampled_ratings.shape[0]), 'w'):
            pass
        
        joint_table = sampled_ratings.merge(vid2seg[['vid', 'index_seg', 'ratio']]\
                             .merge(segs[['index_seg', 'geometry']])\
                             .merge(videos[['vid','URL']]))
        joint_table.ratio = joint_table.ratio.astype(float) 
        joint_table.score = joint_table.score.astype(float)
        y = joint_table.groupby('index_seg').apply(lambda x: (x.score* x.ratio).sum()/x.ratio.sum()).to_frame()
        y.columns = ['ugr']
        y.to_csv(file_name)

In [ ]:
# joint_table = sampled_ratings.merge(vid2seg[['vid', 'index_seg', 'ratio']]\
#                      .merge(segs[['index_seg', 'geometry']])\
#                      .merge(videos[['vid','URL']]))
# joint_table.ratio = joint_table.ratio.astype(float) 
# joint_table.score = joint_table.score.astype(float)
# y = joint_table.groupby('index_seg').apply(lambda x: (x.score* x.ratio).sum()/x.ratio.sum()).to_frame()
# y.columns = ['ugr']
# y.to_csv(file_name)

In [ ]:
# %matplotlib inline

In [ ]:
# y.ugr.hist()